In [1]:
  import json
  import requests
  PTH_DATA = "/content/domain.json"
  PTH_DOMAIN = "/content/train-domain.pddl"
  PTH_PROBLEM = "/content/problem.pddl"

In [2]:
'''This class creates a JSON file inseetirng data, needed to create a pddl problem file'''
class problem_generator:

  def __init__(self):

    self.trains = ""
    self.stations = ""
    self.rails = ""
    self.init_trains_in = ""
    self.init_trains_out = ""
    self.goal_trains_in = ""
    self.goal_trains_out = ""

  '''Save data into a JSON file'''
  def save_file(self):

    problem = {"TRAINS": self.trains,
               "STATIONS": self.stations,
               "PTH_BTW_STATIONS": self.rails,
               "TRAINS_IN_STATION_INIT": self.init_trains_in,
               "TRAINS_OUT_STATION_INIT": self.init_trains_out,
               "TRAINS_IN_STATION_GOAL": self.goal_trains_in,
               "TRAINS_OUT_STATION_GOAL": self.goal_trains_out
               }

    with open(PTH_DATA, "w") as outfile:
      json.dump(problem, outfile)

  '''Insertion of data'''
  def insert_input(self):

    print("INSERT DATA, ATTENTION: FOLLOW STRICTLY THE SUGGESTED SYNTAX, OTHERWISE THE PROGRAM CAN'T RUN")

    self.trains = input("INSERT TRAINS:\n"
                        "(name 1-name 2-...)\n"
                        "(ex. 'T1-T2-...')\n")
    print("\n")
    
    self.stations = input("INSERT STAIONS:\n"
                          "(name 1,num raylways-name2,num raylways-...)\n" 
                          "(ex. 'A,3-B,1-...')\n")
    print("\n")

    self.rails = input("INSERT ALL POSSIBLE LINKS BETWEEN STATION AND NODES, SPECIFYING FOR EACH STATION THE NUMBER OF RAILWAY," 
                       "EACH PATH HAS TO BE SEPARETED BY A ';' :\n"
                       "(nameStation_numRailway-node1-node2-...-nameStation_numRailway; ...)\n"
                       "(ex.: 'A_1-N1-N2-N3-N4-B_1;A_2-N1-...')\n")
    print("\n")
    
    self.init_trains_in = input("INSERT THE INITIAL POSITION OF TRAIN STOPPED IN A STATION:\n"
                                "(nameTrain,nameStation_numRailway- ...)\n "
                                "(ex. 'T1,A_1-T2,B_1-...')\n")
    print("\n")
    
    self.init_trains_out = input("INSERT THE INITIAL POSITION OF TRAIN STOPPED IN A RAILWAY, BETWEEN TWO NODES:\n"
                                "(nameTrain,node1,node2- ...)\n "
                                "(ex. 'T3,N2,N3-...')\n")
    print("\n")
    
    self.goal_trains_in = input("INSERT THE FINAL POSITION OF TRAIN STOPPED IN A STATION:\n"
                                "(nameTrain,nameStation- ...)\n"
                                "(ex. 'T1,A-T2,B-...')\n")
    print("\n")
    
    self.goal_trains_out = input("INSERT THE FINAL POSITION OF TRAIN STOPPED IN A RAILWAY, BETWEEN TWO NODES:\n"
                                "(nameTrain,node1,node2- ...)\n"
                                "(ex. 'T3,N2,N3-...')\n")
    print("\n DONE \n")

In [3]:
d = problem_generator()

In [4]:
d.insert_input()

INSERT DATA, ATTENTION: FOLLOW STRICTLY THE SUGGESTED SYNTAX, OTHERWISE THE PROGRAM CAN'T RUN
INSERT TRAINS:
(name 1-name 2-...)
(ex. 'T1-T2-...')
T1-T2-T3-T4


INSERT STAIONS:
(name 1,num raylways-name2,num raylways-...)
(ex. 'A,3-B,1-...')
A,3-B,1


INSERT ALL POSSIBLE LINKS BETWEEN STATION AND NODES, SPECIFYING FOR EACH STATION THE NUMBER OF RAILWAY,EACH PATH HAS TO BE SEPARETED BY A ';' :
(nameStation_numRailway-node1-node2-...-nameStation_numRailway; ...)
(ex.: 'A_1-N1-N2-N3-N4-B_1;A_2-N1-...')
A_1-N1-N2-N3-N4-B_1;A_2-N1-N2-N3-N4-B_1;A_3-N5-N6-N3-N4-B_1;A_2-N1-N2-N5-N6-N3-N4-B_1


INSERT THE INITIAL POSITION OF TRAIN STOPPED IN A STATION:
(nameTrain,nameStation_numRailway- ...)
 (ex. 'T1,A_1-T2,B_1-...')
T1,A_1-T2,B_1-T4,A_2


INSERT THE INITIAL POSITION OF TRAIN STOPPED IN A RAILWAY, BETWEEN TWO NODES:
(nameTrain,node1,node2- ...)
 (ex. 'T3,N2,N3-...')
T3,N2,N3


INSERT THE FINAL POSITION OF TRAIN STOPPED IN A STATION:
(nameTrain,nameStation- ...)
(ex. 'T1,A-T2,B-...')
T1,B-T2,A-

In [5]:
d.save_file()

In [6]:
class planner:

  def __init__(self, pthfile):
    self.data = json.load(open(pthfile))
    self.trains = []
    self.stations = []
    self.raylways = []
    self.busy_rail = []
    self.objects = ""
    self.init = ""
    self.goal = ""
    self.dict_raylways = {}
    self.num_raylways = 1;
    self.read_objects()
    self.read_init()
    self.read_goal()

  '''Generate a string with the pddl code of the problem from data'''
  def generate_pddl(self):

    define = "(define (problem train-problem)"
    domain = "(:domain train-domain)"
    last_line =")"

    pddl = define+domain+self.objects+self.init+self.goal+last_line

    return pddl

  '''save the pddl code into a file'''
  def save_pddl(self,pth='/content/problem.pddl'):
    pddl = open(pth,"w")
    pddl.write(self.generate_pddl())
    pddl.close()

  '''connect to a server (passing the pddl files as input) which response with a plan (if it exist)'''
  def find_plan(self):
    data = {'domain': open(PTH_DOMAIN, 'r').read(),
            'problem': open(PTH_PROBLEM, 'r').read()}
    resp = requests.post('http://solver.planning.domains/solve', verify=False, json=data).json()

    return resp

  '''read all objects'''
  def read_objects(self):

    self.objects = "(:objects "

    #read trains and insert it in the clause "objects"
    self.trains = self.data["TRAINS"].split("-")
    for t in self.trains:
      self.objects += t
      self.objects += " "

    #read stations and insert it in the clause "objects"
    sts = self.data["STATIONS"].split("-")
    for s in sts:
      app = s.split(",")
      self.stations.append(app[0])
      self.objects += app[0]
      self.objects += " "

    #read raylways between stations and insert it in the clause "objects"
    self.pths_stations = self.data["PTH_BTW_STATIONS"].split(";")
    #read all the paths
    for pth in self.pths_stations:
      p = pth.split("-")
      lenght = len(p)

      #analyze a single path
      for i in range(lenght-1):

        rail = p[i]+" - "+p[i+1]
        inv_rail = p[i+1]+" - "+p[i]

        #case of a rail between a station and a node
        if i == 0:
          new_railway = p[i]
          node = p[i+1]
          rail = new_railway+" - "+node
          inv_rail = node+" - "+new_railway

        #case of a rail between a node and a station
        elif i == lenght-2:
          new_railway = p[i+1]
          node = p[i]
          rail = node+" - "+new_railway
          inv_rail = new_railway+" - "+node

        #case of a rail between two nodes
        else:
          node1 = p[i]
          node2 = p[i+1]
          rail = node1+" - "+node2
          inv_rail = node2+" - "+node1
          if rail not in self.dict_raylways.values() and inv_rail not in self.dict_raylways.values():
            new_railway = "R_"+str(self.num_raylways)
            self.num_raylways+=1

        #chek in railways or rails alredy inserted
        if new_railway not in list(self.dict_raylways.keys()) or rail not in self.dict_raylways.values() and inv_rail not in self.dict_raylways.values():
          self.dict_raylways.update({new_railway:rail})
          self.objects += new_railway
          self.objects += " "

    self.objects += ") "

  '''read initialization of the problem'''
  def read_init(self):

    self.init = "(:init "

    #init trains
    for t in self.trains:
      train = "(train "+t+")"
      self.init += train

    #init stations
    for s in self.stations:
      station = "(station "+s+")"
      self.init += station

    #init railways
    for r in list(self.dict_raylways.keys()):
      railway = "(railway "+r+")"
      self.init += railway

    #init railways of each station
    for station in self.data["STATIONS"].split("-"):
      s = station.split(",")[0]
      num_r = station.split(",")[1]
      for i in range(int(num_r)):
        isin = "(isin "+s+"_"+str(i+1)+" "+s+")"
        self.init+=isin

    #init all the paths
    inv_dict_rails = self.invert_dictionary(self.dict_raylways)
    done = []
    for pth in self.pths_stations:
      p = pth.split("-")
      lenght = len(p)

      #init fluent "conn"
      for i in range(lenght-2):
        rail1 = p[i]+" - "+p[i+1]
        railway1 = inv_dict_rails.get(rail1)
        rail2 = p[i+1]+" - "+p[i+2]
        railway2 = inv_dict_rails.get(rail2)
        rail = railway1+" "+railway2
        inv_rail = railway2+" "+railway1
        if rail not in done and inv_rail not in done:
          done.append(rail)
          done.append(inv_rail)
          conn = "(conn "+rail+")"
          self.init += conn
          inv_conn = "(conn "+inv_rail+")"
          self.init += inv_conn

    #init fluent "at" and "instation"
    init_pos_trains = self.data["TRAINS_IN_STATION_INIT"].split("-")
    for pos in init_pos_trains:
      pos = pos.split(",")
      train = pos[0]
      railway = pos[1]
      station = railway.split("_")[0]
      at1 = "(at "+train+" "+station+")"
      self.init += at1
      at2 = "(at "+train+" "+railway+")"
      self.busy_rail.append(railway)
      self.init += at2
      instation = "(instation "+train+")"
      self.init += instation

    #init fluent "at" for initial position of trains out of a station
    init_pos_trains = self.data["TRAINS_OUT_STATION_INIT"].split("-")
    for pos in init_pos_trains:
      pos = pos.split(",")
      train = pos[0]
      rail = pos[1]+" - "+pos[2]
      railway = inv_dict_rails.get(str(rail))
      at = "(at "+train+" "+railway+")"
      self.init += at
      self.busy_rail.append(railway)

    #init fluent "free"
    for railway in self.dict_raylways.keys():
      if railway not in self.busy_rail:
        free = "(free "+railway+")"
        self.init += free

    self.init += ") "

  '''read the goal of the problem'''
  def read_goal(self):

    self.goal = "(:goal (and "

    #goal for trains into stations
    init_pos_trains = self.data["TRAINS_IN_STATION_GOAL"].split("-")
    for pos in init_pos_trains:
      pos = pos.split(",")
      train = pos[0]
      station = pos[1]
      at = "(at "+train+" "+station+")"
      self.goal += at
      instation = "(instation "+train+")"
      self.goal += instation

    #goal for trains out of stations
    inv_rails_dict = self.invert_dictionary(self.dict_raylways)
    init_pos_trains = self.data["TRAINS_OUT_STATION_GOAL"].split("-")
    for pos in init_pos_trains:
      pos = pos.split(",")
      train = pos[0]
      rail = pos[1]+" - "+pos[2]
      railway = inv_rails_dict.get(str(rail))
      at = "(at "+train+" "+railway+")"
      self.goal += at

    self.goal += ") ) "

  #function to swap keys and values of a dictionary
  def invert_dictionary(self,dictionary):
    inv_dict = {}
    for el in dictionary.items():
      inv_dict.update({el[1]:el[0]})

    return inv_dict

In [7]:
p = planner(PTH_DATA)
p.save_pddl()
res = p.find_plan()

if res['status'] == 'ok':
  print("PLAN FOUND!")

  print("\nDetails on search:")
  out = res['result']['output'].split('\n')
  for i in range(2,7):
    print(out[-i])

  print("\nPlan:")
  plan = '\n'.join([act['name'] for act in res['result']['plan']])
  print(plan)

  print("\nDetails of the plan:")
  chars = '()'
  actions = plan.translate(str.maketrans('', '', chars))
  actions = actions.split("\n")
  for a in range(len(actions)):
    print("ACTION N."+str(a+1)+":")
    act = actions[a].split(" ")
    if act[0] == "out":
      print("Train "+str.upper(act[1])+" avaiable on railway "+str.upper(act[3])+" of station "+str.upper(act[2]))
    if act[0] == "enter":
      print("Train "+str.upper(act[1])+" stopped at railway "+str.upper(act[3])+" of station "+str.upper(act[2]))
    if act[0] == "move":
      rail = p.dict_raylways.get(str.upper(act[2]))+"-"+p.dict_raylways.get(str.upper(act[3]))
      chars = " "
      rail = rail.translate(str.maketrans('', '', chars))
      rail = rail.split("-")
      if(rail[1]==rail[2]):
        print("Move train "+str.upper(act[1])+" on rail "+rail[0]+"-"+rail[1]+"-"+rail[3])
      elif rail[0]==rail[3]:
        print("Move train "+str.upper(act[1])+" on rail "+rail[1]+"-"+rail[0]+"-"+rail[2])
    print("\n")
else:
  print("PLAN NOT FOUND!")

PLAN FOUND!

Details on search:
BFS search completed
Plan found with cost: 19
Nodes expanded during search: 104
Nodes generated during search: 434
Total time: 0.092

Plan:
(out t4 a a_2)
(move t4 a_2 r_1)
(move t4 r_1 r_6)
(move t3 r_2 r_1)
(out t2 b b_1)
(move t3 r_1 a_2)
(enter t3 a_2 a)
(out t1 a a_1)
(move t2 b_1 r_3)
(move t2 r_3 r_5)
(move t1 a_1 r_1)
(move t1 r_1 r_2)
(move t1 r_2 r_3)
(move t2 r_5 r_4)
(move t2 r_4 a_3)
(enter t2 a_3 a)
(move t4 r_6 r_4)
(move t1 r_3 b_1)
(enter t1 b_1 b)

Details of the plan:
ACTION N.1:
Train T4 avaiable on railway A_2 of station A


ACTION N.2:
Move train T4 on rail A_2-N1-N2


ACTION N.3:
Move train T4 on rail N1-N2-N5


ACTION N.4:
Move train T3 on rail N3-N2-N1


ACTION N.5:
Train T2 avaiable on railway B_1 of station B


ACTION N.6:
Move train T3 on rail N2-N1-A_2


ACTION N.7:
Train T3 stopped at railway A of station A_2


ACTION N.8:
Train T1 avaiable on railway A_1 of station A


ACTION N.9:
Move train T2 on rail B_1-N4-N3


ACTION N.